# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto natales,-51.7236,-72.4875,-1.46,90,8,1.50,CL,1690147429
1,1,firuzabad,28.8438,52.5707,29.37,18,100,0.76,IR,1690147429
2,2,adamstown,-25.0660,-130.1015,20.37,71,2,8.00,PN,1690147367
3,3,new plymouth,-39.0667,174.0833,13.23,91,100,1.71,NZ,1690147292
4,4,invercargill,-46.4000,168.3500,8.02,95,98,1.00,NZ,1690147364


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
# Configure the map plot
map_plot1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity"*5,
    scale = 0.01,
    color = "City"
)

# Display the map
map_plot1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
criteria_df = city_data_df.loc[(city_data_df['Max Temp'] < 27) & (city_data_df['Max Temp'] > 21) & 
                               (city_data_df['Wind Speed'] < 4.5) & (city_data_df['Cloudiness']==0)]

# Drop any rows with null values
new_df = criteria_df.dropna()

# Display sample data
new_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
64,64,villamontes,-21.2500,-63.5000,25.21,41,0,2.42,BO,1690147437
84,84,turar ryskulov,42.5333,70.3333,25.29,28,0,3.54,KZ,1690147439
194,194,morondava,-20.2833,44.2833,21.22,76,0,4.38,MG,1690147453
251,251,pisco,-13.7000,-76.2167,23.03,60,0,4.12,PE,1690147368
303,303,meadow lakes,61.6247,-149.6011,23.19,60,0,4.12,US,1690147468
377,377,chibougamau,49.9168,-74.3659,25.79,29,0,3.04,CA,1690147476
378,378,lucapa,-8.4192,20.7447,22.27,29,0,1.65,AO,1690147476
529,529,dauphin,51.1494,-100.0502,24.08,64,0,4.12,CA,1690147493


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
df1 = new_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
df1['Hotel Name'] = ''
         
# Display sample data
hotel_df=df1.reset_index(drop=True)
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,64,villamontes,-21.2500,-63.5000,25.21,41,0,2.42,BO,1690147437,
1,84,turar ryskulov,42.5333,70.3333,25.29,28,0,3.54,KZ,1690147439,
2,194,morondava,-20.2833,44.2833,21.22,76,0,4.38,MG,1690147453,
3,251,pisco,-13.7000,-76.2167,23.03,60,0,4.12,PE,1690147368,
4,303,meadow lakes,61.6247,-149.6011,23.19,60,0,4.12,US,1690147468,
5,377,chibougamau,49.9168,-74.3659,25.79,29,0,3.04,CA,1690147476,
6,378,lucapa,-8.4192,20.7447,22.27,29,0,1.65,AO,1690147476,
7,529,dauphin,51.1494,-100.0502,24.08,64,0,4.12,CA,1690147493,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000

params = {
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key    
    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    Lng = hotel_df.loc[index,'Lng']
    Lat = hotel_df.loc[index,'Lat']

    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{Lng},{Lat},{radius}"
    params["bias"] = f"proximity:{Lng},{Lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
villamontes - nearest hotel: Hoterma
turar ryskulov - nearest hotel: Cheap Hotel with AC
morondava - nearest hotel: Hotel Sharon Menabe
pisco - nearest hotel: La Portada
meadow lakes - nearest hotel: No hotel found
chibougamau - nearest hotel: No hotel found
lucapa - nearest hotel: No hotel found
dauphin - nearest hotel: No hotel found


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,64,villamontes,-21.2500,-63.5000,25.21,41,0,2.42,BO,1690147437,Hoterma
1,84,turar ryskulov,42.5333,70.3333,25.29,28,0,3.54,KZ,1690147439,Cheap Hotel with AC
2,194,morondava,-20.2833,44.2833,21.22,76,0,4.38,MG,1690147453,Hotel Sharon Menabe
3,251,pisco,-13.7000,-76.2167,23.03,60,0,4.12,PE,1690147368,La Portada
4,303,meadow lakes,61.6247,-149.6011,23.19,60,0,4.12,US,1690147468,No hotel found
5,377,chibougamau,49.9168,-74.3659,25.79,29,0,3.04,CA,1690147476,No hotel found
6,378,lucapa,-8.4192,20.7447,22.27,29,0,1.65,AO,1690147476,No hotel found
7,529,dauphin,51.1494,-100.0502,24.08,64,0,4.12,CA,1690147493,No hotel found


In [8]:
# just to check data types in hotel_df
hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City_ID     8 non-null      int64  
 1   City        8 non-null      object 
 2   Lat         8 non-null      float64
 3   Lng         8 non-null      float64
 4   Max Temp    8 non-null      float64
 5   Humidity    8 non-null      int64  
 6   Cloudiness  8 non-null      int64  
 7   Wind Speed  8 non-null      float64
 8   Country     8 non-null      object 
 9   Date        8 non-null      int64  
 10  Hotel Name  8 non-null      object 
dtypes: float64(4), int64(4), object(3)
memory usage: 832.0+ bytes


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
map_plot2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    scale = 0.01,
    color = "Hotel Name", 
    hover_cols = ['Hotel Name','Country', 'Max Temp','Humidity']
)

# Display the map
map_plot2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Country,Max Temp,Humidity)